In [4]:
import pandas as pd

#read the dataset with name "Fake_Real_Data.csv" and store it in a variable df
df = pd.read_csv("True.csv")

#print the shape of dataframe
print(df.shape)

#print top 5 rows
df["text"].head(5)

# (21417, 4)
# 0    WASHINGTON (Reuters) - The head of a conservat...
# 1    WASHINGTON (Reuters) - Transgender people will...
# 2    WASHINGTON (Reuters) - The special counsel inv...
# 3    WASHINGTON (Reuters) - Trump campaign adviser ...
# 4    SEATTLE/WASHINGTON (Reuters) - President Donal...
# Name: text, dtype: object

(21417, 4)


0    WASHINGTON (Reuters) - The head of a conservat...
1    WASHINGTON (Reuters) - Transgender people will...
2    WASHINGTON (Reuters) - The special counsel inv...
3    WASHINGTON (Reuters) - Trump campaign adviser ...
4    SEATTLE/WASHINGTON (Reuters) - President Donal...
Name: text, dtype: object

In [6]:
df_true_news = pd.read_csv("True.csv")
df_fake_news = pd.read_csv("Fake.csv")

In [7]:
df_fake_news['label_num'] = 0
df_true_news['label_num'] = 1

In [8]:
# Concatenate fake and true news
df = pd.concat([df_fake_news, df_true_news])

In [9]:
df.head()

# title	text	subject	date	label_num
# 0	Donald Trump Sends Out Embarrassing New Year’...	Donald Trump just couldn t wish all Americans ...	News	December 31, 2017	0
# 1	Drunk Bragging Trump Staffer Started Russian ...	House Intelligence Committee Chairman Devin Nu...	News	December 31, 2017	0
# 2	Sheriff David Clarke Becomes An Internet Joke...	On Friday, it was revealed that former Milwauk...	News	December 30, 2017	0
# 3	Trump Is So Obsessed He Even Has Obama’s Name...	On Christmas day, Donald Trump announced that ...	News	December 29, 2017	0
# 4	Pope Francis Just Called Out Donald Trump Dur...	Pope Francis used his annual Christmas Day mes...	News	December 25, 2017	0

,title,text,subject,date,label_num
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [10]:
#Removing columns which are not required
df = df.drop(["title", "subject","date"], axis = 1)

In [11]:
df.head()
# text	label_num
# 0	Donald Trump just couldn t wish all Americans ...	0
# 1	House Intelligence Committee Chairman Devin Nu...	0
# 2	On Friday, it was revealed that former Milwauk...	0
# 3	On Christmas day, Donald Trump announced that ...	0
# 4	Pope Francis used his annual Christmas Day mes...	0

,text,label_num
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [12]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [14]:
#This will take some time(nearly 45 minutes)
df['vector'] = df['text'].apply(lambda text: nlp(text).vector)  

In [15]:
df.head()

# text	label_num	vector
# 0	Donald Trump just couldn t wish all Americans ...	0	[-1.6387042, -0.027837504, -1.6619873, -0.2017...
# 1	House Intelligence Committee Chairman Devin Nu...	0	[-1.9558355, 0.5587237, -2.036446, 0.40390933,...
# 2	On Friday, it was revealed that former Milwauk...	0	[-1.9058484, -0.04223674, -1.7389339, -0.02855...
# 3	On Christmas day, Donald Trump announced that ...	0	[-1.6265213, 0.14619705, -0.5877981, 0.0992525...
# 4	Pope Francis used his annual Christmas Day mes...	0	[-2.1327376, 1.1017615, -2.4795847, -0.0065187...

,text,label_num,vector
0,Donald Trump just couldn t wish all Americans ...,0,"[-1.6387042, -0.027837504, -1.6619873, -0.2017..."
1,House Intelligence Committee Chairman Devin Nu...,0,"[-1.9558355, 0.5587237, -2.036446, 0.40390933,..."
2,"On Friday, it was revealed that former Milwauk...",0,"[-1.9058484, -0.04223674, -1.7389339, -0.02855..."
3,"On Christmas day, Donald Trump announced that ...",0,"[-1.6265213, 0.14619705, -0.5877981, 0.0992525..."
4,Pope Francis used his annual Christmas Day mes...,0,"[-2.1327376, 1.1017615, -2.4795847, -0.0065187..."


In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.vector.values,
    df.label_num,
    test_size=0.2,
    random_state=2022
)

In [17]:
import numpy as np
# 2 boyutluya çevirmek için
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [18]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler

# vektörel sunumda çıkan - değerler için scaling yapmam lazım

scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_train_2d)
scaled_test_embed = scaler.transform(X_test_2d)


clf = MultinomialNB()
clf.fit(scaled_train_embed, y_train)

MultinomialNB()

In [20]:
from sklearn.metrics import classification_report
y_pred = clf.predict(scaled_test_embed)

print(classification_report(y_test, y_pred))

#    precision    recall  f1-score   support

#            0       0.89      0.85      0.87      4628
#            1       0.85      0.89      0.87      4352

#     accuracy                           0.87      8980
#    macro avg       0.87      0.87      0.87      8980
# weighted avg       0.87      0.87      0.87      8980


              precision    recall  f1-score   support

           0       0.89      0.85      0.87      4628
           1       0.85      0.89      0.87      4352

    accuracy                           0.87      8980
   macro avg       0.87      0.87      0.87      8980
weighted avg       0.87      0.87      0.87      8980



In [21]:
from  sklearn.neighbors import KNeighborsClassifier

#1. creating a KNN model object
clf = KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean')

#2. fit with all_train_embeddings and y_train
clf.fit(X_train_2d, y_train)

#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(X_test_2d)

#4. print the classfication report
print(classification_report(y_test, y_pred))


# precision    recall  f1-score   support

#            0       0.97      0.97      0.97      4628
#            1       0.97      0.97      0.97      4352

#     accuracy                           0.97      8980
#    macro avg       0.97      0.97      0.97      8980
# weighted avg       0.97      0.97      0.97      8980


              precision    recall  f1-score   support

           0       0.97      0.97      0.97      4628
           1       0.97      0.97      0.97      4352

    accuracy                           0.97      8980
   macro avg       0.97      0.97      0.97      8980
weighted avg       0.97      0.97      0.97      8980

